# **TuriCreate Recommender Model**
The following notebook includes an example of data import, model training and validation evaluation.

In [1]:
import turicreate as tc
import pandas as pd
import time
#from skafossdk import *
import common.save_models as sm

In [ ]:
#ska = Skafos()

## **The Data**
There are 2 main data inputs for a recommender model:
- **items**: items that we want to recommend to a given user, ex. apples
- **actions**: actions users have taken on items, ex. John bought apples

In our example here, we have movies (items) and ratings by users on those movies(actions).

In [2]:
%%capture 
actions = tc.SFrame.read_csv('https://s3.amazonaws.com/skafos.example.data/MovieLensDataset/ml-20m/ratings.csv'); 
items = tc.SFrame.read_csv('https://s3.amazonaws.com/skafos.example.data/MovieLensDataset/ml-20m/movies.csv');

In [6]:
pd_actions = actions.to_dataframe()
sample_user_ids = pd.Series(pd_actions['userId'].unique()).sample(25000)
actions = tc.SFrame(pd_actions[pd_actions['userId'].isin(sample_user_ids)])

In [7]:
%%capture 
# split the training and validation sets up
training_data, validation_data = tc.recommender.util.random_split_by_user(actions, 'userId', 'movieId')

In [8]:
%%capture 
# build the recommender
model = tc.recommender.create(training_data, 'userId', 'movieId')

In [9]:
# export to coreml
coreml_model_name = "recommender.mlmodel"
res = model.export_coreml(coreml_model_name)


This model is exported as a custom Core ML model. In order to use it in your
application, you must also include "libRecommender.dylib". For additional
details see:
https://apple.github.io/turicreate/docs/userguide/recommender/coreml-deployment.html


In [ ]:
%%capture
# grab the results of the model
results = model.recommend();

In [ ]:
# print the validation data
validation_data.print_rows(num_rows=10)

In [ ]:
# evaluate the model
model.evaluate(validation_data)


In [ ]:
# export to coreml
coreml_model_name = "recommender.mlmodel"
res = model.export_coreml(coreml_model_name)

# compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

In [ ]:
# save to Skafos
sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
                     compressed_model = compressed_model,
                     permissions = 'public')